# Library Explorer

Open up KerML semantic library and explore how pyMBE renders it.

In [ ]:
import json
from importlib import resources as lib_resources
import pymbe.api as pm

from pymbe.query.metamodel_navigator import \
    get_effective_basic_name, get_effective_lower_multiplicity, get_effective_upper_multiplicity, get_most_specific_feature_type

## Load Libraries

### Look at Example Libraries Base and ScalarValues

Before we start up a model, which will cause elements to resolve and relationships to be connected and risks finding null references, we pre-stage the libraries for reference in the resolution process.

In the below cell, load up the raw data from the JSON format that is used to interact with the v2 API standard. The namespaces have no name but should have stable IDs when using a standard generation scheme.

In [ ]:
libraries_to_load = ["KernelDataTypeLibrary",
                    "KernelFunctionLibrary",
                    "KernelSemanticLibrary"]
libraries_to_load = ["KernelLibraryExpanded", "SystemsLibrary", "DomainLibrary"]

library_raw_data = {}

for lib_to_load in libraries_to_load:
    with lib_resources.path("pymbe.static_data", lib_to_load + ".json") as lib_data:
        ele_raw_set = json.loads(lib_data.read_text(encoding="utf-8"))
        ele_factored_set = []
        for ele_raw in ele_raw_set:
            factored_data_element = dict(ele_raw["payload"].items()) | {
                    "@id": ele_raw["identity"]["@id"]
                }
            ele_factored_set.append(factored_data_element)
        library_raw_data.update({lib_to_load: ele_factored_set})

In [ ]:
namespace_ids = []

for lib_to_load in libraries_to_load:
    nses = [ele for ele in library_raw_data[lib_to_load] if ele["@type"] == "Namespace"]
    for ns in nses:
        namespace_ids.append(ns["@id"])

namespace_ids

### Cross-references and IDs

To check for all the cross-references, we can look at all fields that refer to some other id in the set of elements.

In [ ]:
def find_all_references_on_element(ele):
    ref_set = set()
    for k, v in ele.items():
        if isinstance(v, dict):
            if "@id" in v:
                ref_set.add(v["@id"])
        if isinstance(v, list):
            if len(v) > 0 and isinstance(v[0], dict):
                for v_item in v:
                    if "@id" in v_item:
                        ref_set.add(v_item["@id"])
    return ref_set

In [ ]:
def find_all_references_in_file(raw_data):
    total_set = set()
    for ele in raw_data:
        total_set = total_set | find_all_references_on_element(ele)
        
    return total_set

In [ ]:
all_refs = set()
all_ids = set()

for lib_to_load in libraries_to_load:
    all_refs = all_refs | find_all_references_in_file(library_raw_data[lib_to_load])
    all_ids = all_ids | {ele["@id"] for ele in library_raw_data[lib_to_load]}

(len(all_refs), len(all_ids), len(all_refs - all_ids))

## Open Library Data

The library data are organized around namespaces, where each namespace corresponds to a specific file in the library.

In [ ]:
library_model = None

with lib_resources.path("pymbe.static_data", "SystemsLibrary.json") as lib_data1:
    with lib_resources.path("pymbe.static_data", "KernelLibraryExpanded.json") as lib_data2:
        with lib_resources.path("pymbe.static_data", "DomainLibrary.json") as lib_data3:
            library_model = pm.Model.load_from_mult_post_files([lib_data1, lib_data2, lib_data3])

In [ ]:
library_pkgs = [pkg for ns in library_model.ownedElement for pkg in ns.throughOwningMembership]
library_pkgs

In [ ]:
parts = [lib_pkg for lib_pkg in library_pkgs if lib_pkg.declaredName == "Parts"][0]
parts

In [ ]:
part = parts.throughOwningMembership[2]

In [ ]:
part.feature

In [ ]:
actions = [lib_pkg for lib_pkg in library_pkgs if lib_pkg.declaredName == "Actions"][0]
actions

In [ ]:
action = actions.throughOwningMembership[-1]
action

In [ ]:
action.feature

## Find Library Namespaces

When the library is exported via the KerML2JSON methods from the reference implementation, a single file is created that contains multiple namespaces, each one representing one of the core library files.

In [ ]:
library_model.ownedElement

## Base Library elements

Load up the Base namespace and get to the root Package within it.

In [ ]:
base_ns = [library_model_ns for library_model_ns in library_model.ownedElement if library_model_ns.throughOwningMembership[0].declaredName == 'Base'][0]

In [ ]:
base_ns.throughOwningMembership[0]._data

### Root Elements of Base library

Look at all the root level elements within the Base package.

In [ ]:
base_ns.throughOwningMembership[0].throughOwningMembership

In [ ]:
base_ns.throughOwningMembership[0].throughOwningMembership[1]._metatype

### Explore multiplicity range objects

Inspect the zeroOrOne multiplicity and look at how multiplicity range elements and their children work together for an optional Feature.

This section looks at both the raw data (data field) and data added by PyMBE libraries (derived field)

In [ ]:
base_ns.throughOwningMembership[0].throughOwningMembership[6]

In [ ]:
base_ns.throughOwningMembership[0].throughOwningMembership[6]._derived

In [ ]:
base_ns.throughOwningMembership[0].throughOwningMembership[6].throughOwningMembership[0].ownedRelationship[0]._data

## Links Library elements

Open up the Links library, which allows for some exploration of basic associations in the library.

In [ ]:
links_ns = [library_model_ns for library_model_ns in library_model.ownedElement if library_model_ns.throughOwningMembership[0].declaredName == 'Links'][0]

In [ ]:
links_ns.throughOwningMembership[0]._data

Look at root level elements.

In [ ]:
links_ns.throughOwningMembership[0].throughOwningMembership

Look at the Features under the BinaryLink.

In [ ]:
links_ns.throughOwningMembership[0].throughOwningMembership

In [ ]:
links_ns.throughOwningMembership[0].throughOwningMembership[2].throughFeatureMembership

In [ ]:
links_ns.throughOwningMembership[0].throughOwningMembership[2].throughFeatureMembership[2]._data

In [ ]:
links_ns.throughOwningMembership[0].throughOwningMembership[2]._data

Examine the FeatureMembership from BinaryLink to participant Feature (and see that there is a memberName here as well as the Feature declared name).

In [ ]:
links_ns.throughOwningMembership[0].throughOwningMembership[2].ownedRelationship[2]._data

In [ ]:
links_ns.throughOwningMembership[0].throughOwningMembership[2].throughFeatureMembership

In [ ]:
links_ns.throughOwningMembership[0].throughOwningMembership[2].throughFeatureMembership[0]._derived

In [ ]:
links_ns.throughOwningMembership[0].throughOwningMembership[2].throughFeatureMembership[0].throughRedefinition

In [ ]:
links_ns.throughOwningMembership[0].throughOwningMembership[2].throughFeatureMembership[0]._data

In [ ]:
links_ns.throughOwningMembership[0].throughOwningMembership[2].throughFeatureMembership[0].throughRedefinition[0]._data

## Performance Library elements

Open up the Performance library, which allows for some exploration of redefinition and interpreting some implicit Feature elements (name, multiplicity, type) that are important for model interpretation.

In [ ]:
peform_ns = [library_model_ns
               for library_model_ns in library_model.ownedElement
               if library_model_ns.throughOwningMembership[0].declaredName == 'Performances'][0]

In [ ]:
peform_ns.throughOwningMembership[0].throughOwningMembership

In [ ]:
peform_ns.throughOwningMembership[0].ownedRelationship

In [ ]:
perform_eles = peform_ns.throughOwningMembership[0].throughOwningMembership

performance = None
    
for perform_ele in perform_eles:
    if perform_ele._metatype in ('Behavior'):
        if hasattr(perform_ele, "declaredName"):
            if perform_ele.declaredName == 'Performance':
                performance = perform_ele
                
performance

### Performance library element exploration

Look witin the Performance object and its Features that redefine other Features (from Occurrences for example). 

In [ ]:
performance.throughFeatureMembership

In [ ]:
performance.throughFeatureMembership[3]._derived

In [ ]:
performance.throughFeatureMembership[3].throughRedefinition[0]

In [ ]:
get_effective_basic_name(performance.throughFeatureMembership[3])

### Vector Values library element exploration

Look around the vector values library element for redefinied multiplicity and type.

In [ ]:
vvals_ns = [library_model_ns for library_model_ns in library_model.ownedElement if library_model_ns.throughOwningMembership[0].declaredName == 'VectorValues'][0]
vvals_ns

In [ ]:
vvals_eles = vvals_ns.throughOwningMembership[0].throughOwningMembership
vvals_eles

In [ ]:
vvals_eles[4]

In pyMBE, the implicit names (names from redefined Features) are represented with the redefines shorthand ( :>> ).

In [ ]:
vvals_eles[4].throughFeatureMembership[0]

In [ ]:
get_effective_basic_name(vvals_eles[4].throughFeatureMembership[0])

In [ ]:
vvals_eles[4].throughFeatureMembership[0]

In [ ]:
get_most_specific_feature_type(vvals_eles[4].throughFeatureMembership[0])

In [ ]:
get_effective_lower_multiplicity(vvals_eles[4].throughFeatureMembership[0])

In [ ]:
get_effective_upper_multiplicity(vvals_eles[4].throughFeatureMembership[0])

In [ ]:
vvals_eles[4].throughFeatureMembership[0].throughRedefinition[0]

In [ ]:
vvals_eles[4].throughFeatureMembership[0].throughRedefinition[0].throughOwningMembership[0]._metatype

In [ ]:
vvals_eles[4].throughFeatureMembership[0].throughRedefinition[0].throughOwningMembership[0].throughOwningMembership

In [ ]:
vvals_eles[4].throughFeatureMembership[0].throughFeatureTyping

In [ ]:
vvals_eles[4].throughFeatureMembership[0].throughRedefinition

### Vector Functions library element exploration

Look around the vector function library elements for the rendering of expression trees. 

In [ ]:
vfunc_ns = [library_model_ns
               for library_model_ns in library_model.ownedElement
               if library_model_ns.throughOwningMembership[0].declaredName == 'VectorFunctions'][0]

In [ ]:
vfunc_ns.throughOwningMembership[0].throughOwningMembership

Show the invariants as features within the adding ( + ) function rendered by following the expressions.

In [ ]:
vfunc_ns.throughOwningMembership[0].throughOwningMembership[3].throughFeatureMembership

Drill more down into the parts of the larger expressions.

In [ ]:
vfunc_ns.throughOwningMembership[0].throughOwningMembership[3].throughFeatureMembership[2].throughResultExpressionMembership[0]._data

In [ ]:
vfunc_ns.throughOwningMembership[0].throughOwningMembership[3].throughFeatureMembership[2].throughResultExpressionMembership[0]._derived

In [ ]:
vfunc_ns.throughOwningMembership[0].throughOwningMembership[3].throughFeatureMembership[2].\
    throughResultExpressionMembership[0].throughParameterMembership[1]._derived

In [ ]:
fre_trial = vfunc_ns.throughOwningMembership[0].throughOwningMembership[3].throughFeatureMembership[2].\
    throughResultExpressionMembership[0].throughParameterMembership[1].throughFeatureValue[0]
fre_trial

In [ ]:
fre_trial._derived

In [ ]:
fre_trial.throughFeatureMembership

In [ ]:
vfunc_ns.throughOwningMembership[0].throughOwningMembership[3].throughFeatureMembership[2].throughReturnParameterMembership